In [1]:

import sys
import shutil
import pdfplumber
import os
import re

# 🥈 Supprimer complètement les erreurs
sys.stderr = open(os.devnull, 'w')

# === Fonctions Utilitaires ===

def clean_value(value):
    return ''.join(filter(str.isdigit, value)) if value else ""

def extract_data(text, keyword):
    for line in text.split('\n'):
        if keyword in line:
            parts = line.split()
            for i, part in enumerate(parts):
                if keyword in part and i + 1 < len(parts):
                    return parts[i + 1]
    return ""

def extract_auftrag_from_text(text):
    
    # Trouve tous les numéros à 7-12 chiffres après le mot "Auftrag"
    return re.findall(r"Auftrag[^0-9]{0,10}(\d{7,12})", text, re.IGNORECASE)


# === Chemins ===

lieferung_folder = r"D:/Work_on_Benz/Würth_Lieferung"
rechnung_folder = r"D:/Work_on_Benz/Würth_Rechnung"
matched_folder = r"D:/Work_on_Benz/Result_folder_Wurth/matched"
unmatched_folder = r"D:/Work_on_Benz/Result_folder_Wurth/unmatched"
os.makedirs(os.path.join(unmatched_folder, "Lieferschein"), exist_ok=True)
os.makedirs(os.path.join(unmatched_folder, "Rechnung"), exist_ok=True)
os.makedirs(matched_folder, exist_ok=True)

# === Extraction des fichiers avec leurs valeurs associées ===

lieferung_map = {}  # {valeur : chemin du fichier}
for f in os.listdir(lieferung_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(lieferung_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    val = extract_auftrag_from_text(text)
                    print(f"[LIEFERSCHEIN] Fichier: {f} → Auftrag extrait: '{val}'")

                    for auftrag in val:
                        lieferung_map[auftrag] = path
                        break

rechnung_map = {}  # {auftrag: [liste de chemins des fichiers rechnung]}
for f in os.listdir(rechnung_folder):
    if f.lower().endswith(".pdf"):
        path = os.path.join(rechnung_folder, f)
        with pdfplumber.open(path) as pdf:
            for page in pdf.pages:
                text = page.extract_text()
                if text:
                    auftrag_extrait = extract_auftrag_from_text(text)
                    print(f"[RECHNUNG] Fichier: {f} → Auftrag extraits: {auftrag_extrait}")
                    for auftrag in auftrag_extrait:
                        rechnung_map.setdefault(auftrag, []).append(path)
                    break

# === Comparaison et tri ===

matched_keys = set(lieferung_map.keys()) & set(rechnung_map.keys())

# Fichiers appariés
for key in matched_keys:
    match_dir = os.path.join(matched_folder, key)
    os.makedirs(os.path.join(match_dir, "Lieferschein"), exist_ok=True)
    os.makedirs(os.path.join(match_dir, "Rechnung"), exist_ok=True)
    
    shutil.copy2(lieferung_map[key], os.path.join(match_dir, "Lieferschein", os.path.basename(lieferung_map[key])))
    
    for rechnung_path in rechnung_map[key]:
        shutil.copy2(rechnung_path, os.path.join(match_dir, "Rechnung", os.path.basename(rechnung_path)))

# Fichiers non appariés
for key, path in lieferung_map.items():
    if key not in matched_keys:
        shutil.copy2(path, os.path.join(unmatched_folder, "Lieferschein", os.path.basename(path)))

used_files = {os.path.basename(p) for key in matched_keys for p in rechnung_map[key]}
for auftrag, paths in rechnung_map.items():
    if auftrag not in matched_keys:
        for path in paths:
            if os.path.basename(path) not in used_files:
                shutil.copy2(path, os.path.join(unmatched_folder, "Rechnung", os.path.basename(path)))

print("Classement terminé 😊 .")


[LIEFERSCHEIN] Fichier: 2025-01-31_1614_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8103778812_Würth_Lieferschein_8103778812.pdf → Auftrag extrait: '['2227555615']'
[LIEFERSCHEIN] Fichier: 2025-01-31_1614_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8103778812_Würth_Lieferschein_8103778812.pdf → Auftrag extrait: '[]'
[LIEFERSCHEIN] Fichier: 2025-02-06_1355_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8104350866_Würth_Lieferschein_8104350866.pdf → Auftrag extrait: '['2228022094']'
[LIEFERSCHEIN] Fichier: 2025-02-06_1355_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8104350866_Würth_Lieferschein_8104350866.pdf → Auftrag extrait: '[]'
[LIEFERSCHEIN] Fichier: 2025-02-06_1827_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8104396799_Würth_Lieferschein_8104396799.pdf → Auftrag extrait: '['2228065043']'
[LIEFERSCHEIN] Fichier: 2025-02-06_1827_Würth_Ihr_digitaler_Lieferschein_für_Lieferung_8104396799_Würth_Lieferschein_8104396799.pdf → Auftrag extrait: '[]'
[LIEFERSCHEIN] Fichier: 2025